In [1]:
# ! pip install pandas

### Import Necessary Libraries

In [2]:
import io
import os
import glob
import pandas as pd
from datetime import datetime, timedelta

curr_date = datetime.now().strftime('%d-%m-%Y')
prev_day = (datetime.now() - timedelta(days=1)).strftime('%d-%m-%Y')

print("Current date:", curr_date)
print("Previous day:", prev_day)

Current date: 20-06-2024
Previous day: 19-06-2024


### Read files into Dataframes

In [3]:
column_names = ["S/N", "CHANNEL", "SESSION ID", "TRANSACTION TYPE", "RESPONSE", "AMOUNT",
                "TRANSACTION TIME", "ORIGINATOR INSTITUTION", "ORIGINATOR / BILLER",
                "DESTINATION INSTITUTION", "DESTINATION ACCOUNT NAME", "DESTINATION ACCOUNT NO",
                "NARRATION", "PAYMENT REFERENCE", "LAST 12 DIGITS OF SESSION_ID"]

categories = ["BILLING", "CMMS", "EBILLSPAY", "NIBSS_EASY_PAY", "NIBSS_NIP_POS", "NIBSS_NIP_EBILLSPAY",
              "NIP", "NIP DISPUTE", "NIP REVERSAL", "NQR_MERCHANT", "PRODUCTS", "NIBSS_NIP_GSI"]

session = []

dtype_mapping = {5: str, 11: str}

for folder_path in glob.glob("../Files/*/"):
    folder_name = os.path.basename(os.path.normpath(folder_path))
    session.append(folder_name)

dfs = []

### Data Cleaning and Transformation

In [4]:
for session_folder in session:
    print(f"Session Folder: {session_folder}")
    for category in categories:
        file_pattern = f"../Files/{session_folder}/{category}/*outwards successful.csv"
        file_paths = glob.glob(file_pattern)
        # print(f"File pattern for '{category}' under '{session_folder}': {file_pattern}")

        for file_path in file_paths:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                df = pd.read_csv(io.StringIO(content), header=None, skiprows=6, names=column_names, dtype=dtype_mapping)
                df = df.dropna(subset=df.columns[8:15], how='all')
                df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
                df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
                df["category"] = category
                df["session"] = session_folder
                dfs.append(df)
                # print(f"Appended DataFrame for file: {file_path}")
            except pd.errors.ParserError:
                print(f"Error parsing file: {file_path}. Please check the file structure.")
            except UnicodeDecodeError:
                print(f"Encoding error for file: {file_path}. Attempting to convert to UTF-8...")
                try:
                    with open(file_path, 'r', encoding='latin-1') as file:
                        content = file.read()
                    df = pd.read_csv(io.StringIO(content), header=None, skiprows=6, names=column_names, dtype=dtype_mapping)
                    df = df.dropna(subset=df.columns[8:15], how='all')
                    df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
                    df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
                    df["category"] = category
                    df["session"] = session_folder
                    dfs.append(df)
                    print(f"Successfully converted and appended DataFrame for file: {file_path}")
                except Exception as e:
                    print(f"Failed to convert file: {file_path}. Error: {str(e)}")

combined_df = pd.concat(dfs, ignore_index=True)
combined_df["S/N"] = range(1, len(combined_df) + 1)
print("Combined DataFrame is ready")

Session Folder: ECOBANK_20240619_055959
Session Folder: ECOBANK_20240619_085959
Session Folder: ECOBANK_20240619_095959
Encoding error for file: ../Files/ECOBANK_20240619_095959/NIP\NIP_050_outwards successful.csv. Attempting to convert to UTF-8...
Successfully converted and appended DataFrame for file: ../Files/ECOBANK_20240619_095959/NIP\NIP_050_outwards successful.csv
Session Folder: ECOBANK_20240619_105959
Session Folder: ECOBANK_20240619_115959
Session Folder: ECOBANK_20240619_125959
Session Folder: ECOBANK_20240619_135959
Session Folder: ECOBANK_20240619_145959
Session Folder: ECOBANK_20240619_165959
Session Folder: ECOBANK_20240619_185959
Session Folder: ECOBANK_20240619_205959
Session Folder: ECOBANK_20240619_235959
Encoding error for file: ../Files/ECOBANK_20240619_235959/NIP\NIP_050_outwards successful.csv. Attempting to convert to UTF-8...
Successfully converted and appended DataFrame for file: ../Files/ECOBANK_20240619_235959/NIP\NIP_050_outwards successful.csv
Combined Dat

In [5]:
# # Iterate through each folder in the session list
# for session_folder in session:
#     print(f"Session Folder: {session_folder}")
#     for category in categories:
#         file_pattern = f"../Files/{session_folder}/{category}/*outwards successful.csv"
#         file_paths = glob.glob(file_pattern)
#         print(f"File pattern for '{category}' under '{session_folder}': {file_pattern}")

#         for file_path in file_paths:
#             try:
#                 df = pd.read_csv(file_path, header=None, skiprows=6, names=column_names, encoding='utf-8', dtype=dtype_mapping)
#                 df = df.dropna(subset=df.columns[8:15], how='all')
#                 df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
#                 df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
#                 df["category"] = category
#                 df["session"] = session_folder
#                 dfs.append(df)
#                 print(f"Appended DataFrame for file: {file_path}")
#             except pd.errors.ParserError:
#                 print(f"Error parsing file: {file_path}. Please check the file structure.")
#             except UnicodeDecodeError:
#                 print(f"Encoding error for file: {file_path}. Ignored.")

# combined_df = pd.concat(dfs, ignore_index=True)
# combined_df["S/N"] = range(1, len(combined_df) + 1)
# print("Combined DataFrame is ready")

### Display Cleaned Data

In [6]:
display(combined_df)

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session
0,1,THIRD-PARTY PAYMENT PLATFORM,'999999240619001356323764028923,Fund Transfer (Direct Credit),Approved or Completed Successfully,50.00,2024-06-19 00:14:00,ECOBANK,ANIFOWOSE TAWAKALITU KIKELOMO,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-3511260319,'NIPMINI/110006240619001355000000979454/PSTK/T...,'323764028923,NIBSS_EASY_PAY,ECOBANK_20240619_055959
1,2,THIRD-PARTY PAYMENT PLATFORM,'999999240619024307618012585918,Fund Transfer (Direct Credit),Approved or Completed Successfully,2000.00,2024-06-19 02:43:10,ECOBANK,ADENIYAN ADETUTU BLESSING,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-4263005818,'NIPMINI/110006240619024306000000979983/PSTK/T...,'618012585918,NIBSS_EASY_PAY,ECOBANK_20240619_055959
2,3,THIRD-PARTY PAYMENT PLATFORM,'999999240619024544356670058686,Fund Transfer (Direct Credit),Approved or Completed Successfully,1000.00,2024-06-19 02:45:47,ECOBANK,OMOREGBEE WINNIE EDUGIE,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-4400000562,'NIPMINI/110006240619024543000000980001/PSTK/T...,'356670058686,NIBSS_EASY_PAY,ECOBANK_20240619_055959
3,4,MOBILE PHONE,'999999240619010040790943375062,Fund Transfer (Direct Credit),Approved or Completed Successfully,812.03,2024-06-19 01:00:44,ECOBANK,ABRAHAM DESTINY SIMON,ZENITH BANK,NEWEDGE FINANCE LID NIP,1223458380,Payment to 1223458380 from 2341243998,'NIPMINI/050004240619010040144127660041/xc7234...,'790943375062,NIBSS_EASY_PAY,ECOBANK_20240619_055959
4,5,MOBILE PHONE,'999999240619010043479453851645,Fund Transfer (Direct Credit),Approved or Completed Successfully,13207.41,2024-06-19 01:00:45,ECOBANK,NATHAN TSAGA,ZENITH BANK,NEWEDGE FINANCE LID NIP,1223458380,Payment to 1223458380 from 2621202369,'NIPMINI/050004240619010042153749397533/xc7234...,'479453851645,NIBSS_EASY_PAY,ECOBANK_20240619_055959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299832,299833,MOBILE PHONE,'000010240609152454111305636717,Fund Transfer (Direct Credit),Approved or Completed Successfully,39500.00,2024-06-09 16:24:58,ECOBANK,MUSBAHU USMAN BALA,GTBANK,BELLO KABIR,0426421404,REFNGA1378446199 BO MUSBAHU USMAN BALA IFO BELLO,'NGA1378446199,'111305636717,NIP REVERSAL,ECOBANK_20240619_235959
299833,299834,MOBILE PHONE,'000010240609155431111190902400,Fund Transfer (Direct Credit),Approved or Completed Successfully,3000.00,2024-06-09 16:54:35,ECOBANK,ONYINYE UMEONYIDO,GTBANK,ANYAEGBU CHIDINMA PRECIOUS,0237334122,REFNGA1378509329 BO ONYINYE UMEONYIDO IFO ANYAEGB,'NGA1378509329,'111190902400,NIP REVERSAL,ECOBANK_20240619_235959
299834,299835,MOBILE PHONE,'000010240610165549111173757266,Fund Transfer (Direct Credit),Approved or Completed Successfully,9700.00,2024-06-10 17:55:53,ECOBANK,IGWEMOH LILIAN,GTBANK,IGWEMOH LILIAN LADIDI,0695269204,REFNGA1381039483 BO IGWEMOH LILIAN IFO IGWEMOH LI,'NGA1381039483,'111173757266,NIP REVERSAL,ECOBANK_20240619_235959
299835,299836,MOBILE PHONE,'000010240610213238111098038365,Fund Transfer (Direct Credit),Approved or Completed Successfully,10000.00,2024-06-10 22:32:42,ECOBANK,OLADIMEJI LATEEF,GTBANK,LATEEF OLADIMEJI HAKEEM,7089823709,REFNGA1381682008 BO OLADIMEJI LATEEF IFO LATEEF O,'NGA1381682008,'111098038365,NIP REVERSAL,ECOBANK_20240619_235959


### Write Data back into the Output Directory

In [7]:
output_dir = f"../Output/{prev_day}"
os.makedirs(output_dir, exist_ok=True)

In [8]:
output_file_path = os.path.join(output_dir, f"Outwards successful_{prev_day}.csv")
# output_file_path = os.path.join(output_dir, f"Outwards successful_17-05-2024.csv")
combined_df.to_csv(output_file_path, index=False, mode='w')

print(f"Output file saved to: {output_file_path}")

Output file saved to: ../Output\Outwards successful_19-06-2024.csv
